In [1]:
import torch

from dataProcessing.ballerDataset import BallerDataset
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from models.macroModel import BallerModel
from engineMacro import train,test

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Le haiperparameteros


In [3]:
IN_SEQ_LEN= 25
OUT_SEQ_LEN = 15
PLAYER_BALL_ENCODING_SIZE = 32   #this number *2 must be divisible by n_heads
DROPOUT = 0.2
LR = 1e-3
BATCH_SIZE = 512
EPOCHS = 10

USE_LR_SCHEDULER = True
SCHEDULER_PATIENCE = 10

## Create DataLoaders


In [4]:
train_path = '../test/train/'
validation_path = '../test/validation/'
test_path = '../test/test/'


train_dataset = BallerDataset(train_path)
validation_dataset = BallerDataset(validation_path)
test_dataset = BallerDataset(test_path)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


## Create Model and things

In [5]:
model = BallerModel(player_ball_encoding_size=PLAYER_BALL_ENCODING_SIZE,
                    dropout=DROPOUT)
model = model.double()
model = model.to(device)
loss_func = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr = LR)
scheduler = ReduceLROnPlateau(optim, mode='min', patience=SCHEDULER_PATIENCE, verbose=True) if USE_LR_SCHEDULER else None

In [6]:
tgt_mask = BallerModel.generate_square_subsequent_mask(
    dim1=OUT_SEQ_LEN,
    dim2=OUT_SEQ_LEN
    )
src_mask = BallerModel.generate_square_subsequent_mask(
    dim1=OUT_SEQ_LEN,
    dim2=IN_SEQ_LEN
    )
tgt_mask = tgt_mask.to(device)
src_mask = src_mask.to(device)

## Train Model

In [ ]:
model, train_losses, val_losses = train(model = model,
      loss_func=loss_func,
      optim=optim,
      scheduler=scheduler,
      dataloaders=(train_dataloader,validation_dataloader),
      epochs=EPOCHS,
      device=device,
      src_mask=src_mask,
      tgt_mask=tgt_mask,
      verbose=True
)

################
#Begin training#
################
Epoch:0  Train Loss:0.04836  Val Loss:0.03218, Total Time:527.727680683136, Epoch Time:527.727680683136
Epoch:1  Train Loss:0.02781  Val Loss:0.02544, Total Time:1057.9251368045807, Epoch Time:530.1974561214447
